# 데이터 로딩 및 확인

In [8]:
import glob, os 
import pandas as pd

all_files = glob.glob(os.path.join(r'.\데이터셋\topics','*.data'))
print(type(all_files))
print(all_files[:2])

df = pd.read_table(all_files[0], encoding='latin1')
print(df.shape)
print(df.head())
#print(df.to_string)

<class 'list'>
['.\\데이터셋\\topics\\performance_honda_accord_2008.txt.data', '.\\데이터셋\\topics\\parking_bestwestern_hotel_sfo.txt.data']
(50, 1)
  Very happy with my 08 Accord, performance is quite adequate it has nice looks and is a great long,  distance cruiser .
0   6, 4, 3 eco engine has poor performance and g...                                                                    
1   Overall performance is good but comfort level...                                                                    
2   I'm impressed with the performance as well as...                                                                    
3   It has room, performance, good MPG for its si...                                                                    
4   For the record I test, drove the Lexus350 the...                                                                    


In [11]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_row',None)
pd.set_option('display.max_colwidth',None)

In [38]:
path = r'.\데이터셋\topics'
all_files = glob.glob(os.path.join(r'.\데이터셋\topics','*.data'))
filenames =[]
opinion_text = []
for file in all_files:
    df = pd.read_table(file, encoding='latin1')
    filename = file.split('\\')[-1]
    filename = filename.split('.')[0]
    filenames.append(filename)
    opinion_text.append(df.to_string())
document_df = pd.DataFrame({'filename':filenames,'opinion_text':opinion_text})
print(document_df.shape)
display(document_df.head().iloc[0])

(51, 2)


filename                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

# TfidVectorizer의 tokenizer 인자로 사용될 어근 변환 함수 선언

In [39]:
print(string.punctuation)
print(ord('A'))
print(chr(65))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
65
A


- Translate 함수
>- 문자열 내에 특정 문자를 다른 문자로 일괄 변경하는 함수
>- 매개변수로 변경하고자 하는 정보를 딕셔너리로 전달한다 {원본문자: 변경문자 유니코드}

In [40]:
trans = str.maketrans('[],','   ')
print(trans)
text = '[고양이, 강아지, 다람쥐, 토끼]'
text = text.translate(trans)
print(text)

{91: 32, 93: 32, 44: 32}
 고양이  강아지  다람쥐  토끼 


In [41]:
from nltk.stem import WordNetLemmatizer
import nltk
import string
# ord(): 문자 -> 유니코드(숫자)를 반환
# chr() : 유니코드(숫자) -> 문자
remove_punct_dict = {ord(punct):None for punct in string.punctuation}

# 입력된 문장을 받아서 소문자로 변환 -> 구두점 제거 -> 단어 토큰화 -> 어근변환

def LemNormalize(text):
    tokens = nltk.word_tokenize(text.lower().translate(remove_punct_dict))
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]


# TF-IDF 기반 피처 벡터화
- min_df : 전체 문서에 걸쳐 너무 낮은 빈도를 갖는 단어 피처를 제외
  >- 정수 : 지정된 개수 이하로 나타나는 단어는 피처 추출제외
  >- 실수 (0.0~1.0) : 지정한 백분율 이하 빈도수까지의 단어는 피처 추출에서 제외.(ex.0.05 하위 50프로 이하 빈도수를 가지는 단어는 피처 추출에서 제외)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

stopwords = nltk.corpus.stopwords.words('english')
tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords, ngram_range=(1,2), min_df=0.05, max_df=0.85)
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

C:\Users\user\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\user\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doe', 'doesnt', 'dont', 'ha', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wa', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  warnings.warn(


In [43]:
print(feature_vect.shape)
print(feature_vect[0].toarray())

(51, 5014)
[[0. 0. 0. ... 0. 0. 0.]]


# 군집분석

## 군집 수 5개로 군집

In [44]:
from sklearn.cluster import KMeans
km_cluster = KMeans(n_clusters=5, random_state=0, max_iter=10000)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_center = km_cluster.cluster_centers_

C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [45]:
document_df['cluster_label'] = cluster_label
display(document_df.head())

filename  \
0  performance_honda_accord_2008   
1  parking_bestwestern_hotel_sfo   
2      quality_toyota_camry_2007   
3            price_amazon_kindle   
4     performance_netbook_1005ha   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [50]:
display(document_df[document_df['cluster_label']==4]['filename'])

5            price_holiday_inn_london
8              food_swissotel_chicago
11            food_holiday_inn_london
17      service_bestwestern_hotel_sfo
18         service_holiday_inn_london
20    service_swissotel_hotel_chicago
30        location_holiday_inn_london
Name: filename, dtype: object

## 군집 수 3개로 군집

In [58]:
from sklearn.cluster import KMeans
km_cluster = KMeans(n_clusters=5, random_state=0, max_iter=10000)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_center = km_cluster.cluster_centers_
document_df['cluster_label']=cluster_label
display(document_df[document_df['cluster_label']==2]['filename'])

C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


3               price_amazon_kindle
4        performance_netbook_1005ha
6          navigation_amazon_kindle
9               fonts_amazon_kindle
12    eyesight-issues_amazon_kindle
21              sound_ipod_nano_8gb
23         size_asus_netbook_1005ha
31          keyboard_netbook_1005ha
36      screen_garmin_nuvi_255W_gps
37            screen_netbook_1005ha
39             screen_ipod_nano_8gb
42            buttons_amazon_kindle
43      battery-life_netbook_1005ha
44       battery-life_amazon_kindle
45       battery-life_ipod_nano_8gb
Name: filename, dtype: object

# 군집별 핵심 단어 추출하기
- cluster_centers_
  >- 단어 피쳐가 군집의 중심으로 부터 얼마나 가깝게 있는지를 군집 내의 피쳐들과의 상대 위치로 나타낸 값
  >- 0~1 사이의 값으로 1에 가까울 수록 중심과 가까운 것을 의미

In [62]:
from sklearn.cluster import KMeans
cluster_center = km_cluster.cluster_centers_
#3 은 군집수, 5014 피쳐의 수
print(cluster_centers.shape)
print(cluster_center[0][:10])

(5, 5014)
[0.0123956  0.         0.00192668 0.         0.         0.00129598
 0.         0.00130398 0.00279103 0.        ]


## 군집별 상위 n개의 핵심단어 추출함수 정의
- 매개변수 설명
  >- cluster model : KMeans모델 객체
  >- cluter_data : 리뷰 문서 및 군집 결과 값을 가지고 있는 DataFrame 객체
  >- featurenames : 피처 카운터 벡터화 된 각 피쳐(단어)의 이름
  >- cluster num : 군집수
  >- top n features : 군집별 확인하고자 하는 상위 n개의 피쳐

In [74]:
def get_cluster_details(cluster_model, cluster_data, feature_names, cluster_num, top_n_features=10):
    cluster_details = {}
    centroid_feature_ordered_idx = cluster_model.cluster_centers_.argsort()[:,::-1]  
    for num in range(cluster_num) : # 군집 수 만큼 반복 수행
        #{0:{}}
        cluster_details[num] = {} # 개별 군집 정보를 담을 사전 객체를 초기화 
       #{0:{'cluster':0}}
        cluster_details[num]['cluster'] = num
        top_feature_indexes = centroid_feature_ordered_idx [num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes]
        # 군집의 중심으로부터 가까운 상위 n개 단어의 거리값 목록
        top_features_values = cluster_model.cluster_centers_[num,top_feature_indexes].tolist()
        cluster_details[num]['top_features'] = top_features
        cluster_details[num]['top_features_value'] = top_features_values
        filenames = cluster_data[cluster_data['cluster_label']==num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[num]['filenames'] = filenames
    return cluster_details

In [78]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_details in cluster_details.items():
        print(f'#### Cluster:{cluster_num} ####')
        print(f'Top Features:{cluster_details["top_features"]}')
        print(f'Review 파일명:{cluster_details["filenames"][:5]}')
        print('*'*40)

In [79]:
feature_names = tfidf_vect.get_feature_names_out()
cluster_details = get_cluster_details(km_cluster, document_df, feature_names, 3)
print_cluster_details(cluster_details)

#### Cluster:0 ####
Top Features:['direction', 'voice', 'map', 'accurate', 'feature', 'satellite', 'speed', 'speed limit', 'video', 'update']
Review 파일명:['features_windows7', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps', 'speed_garmin_nuvi_255W_gps', 'speed_windows7']
****************************************
#### Cluster:1 ####
Top Features:['room', 'staff', 'hotel', 'bathroom', 'clean', 'parking', 'location', 'free', 'room wa', 'staff wa']
Review 파일명:['parking_bestwestern_hotel_sfo', 'staff_swissotel_chicago', 'location_bestwestern_hotel_sfo', 'rooms_swissotel_chicago', 'rooms_bestwestern_hotel_sfo']
****************************************
#### Cluster:2 ####
Top Features:['screen', 'battery', 'battery life', 'keyboard', 'life', 'kindle', 'size', 'page', 'button', 'font']
Review 파일명:['price_amazon_kindle', 'performance_netbook_1005ha', 'navigation_amazon_kindle', 'fonts_amazon_kindle', 'eyesight-issues_amazon_kindle']
****************************************
